In [18]:
# importing system modules
import arcpy
import os

In [19]:
# setting workspace
arcpy.env.workspace = "H:/ArcGIS/geogfinalproject/Data"
arcpy.env.overwriteOutput = True

In [20]:
# creating a geodatabase
mygdb = "H:/ArcGIS/geogfinalproject/tri.gdb"

In [21]:
# path for the gdb
dir_path = os.path.dirname(os.path.realpath(mygdb))
outgdb = os.path.basename(mygdb)

In [22]:
# actually creating the GDB
arcpy.CreateFileGDB_management(dir_path, outgdb)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb'>

In [23]:
# sending communities to shapefile
outworkspace = mygdb
fcList = arcpy.ListFeatureClasses()

In [24]:
# loop to grab the shapefiles
for shapefile in fcList:
    out_featureclass = os.path.join(outworkspace, os.path.splitext(shapefile)[0])

In [25]:
# sending the shapefile to the gdb
arcpy.CopyFeatures_management(shapefile, out_featureclass)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb\\communityAreas'>

In [26]:
# sending tables to the gdb
tables = arcpy.ListTables()

In [27]:
outLocation = mygdb

In [28]:
arcpy.TableToGeodatabase_conversion(tables, outLocation)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb'>

In [29]:
# setting new workspace
arcpy.env.workspace = mygdb
arcpy.env.overwriteOutput = True

In [30]:
# creating XY points from TRI release
arcpy.management.XYTableToPoint("fac_release_cleared", "TRIpoints", "long", "lat")

<Result 'H:/ArcGIS/geogfinalproject/tri.gdb\\TRIpoints'>

In [31]:
# adding field for join
arcpy.management.AddField("communityAreas", "areaNUM", "LONG")

<Result 'communityAreas'>

In [32]:
# calculating field to join
arcpy.management.CalculateField("communityAreas", "areaNUM", "!area_num_1!")

<Result 'communityAreas'>

In [33]:
#projecting community areas for spatial join
input_features = r"H:/ArcGIS/geogfinalproject/tri.gdb/communityAreas"
output_feature_class = r"H:/ArcGIS/geogfinalproject/tri.gdb/communityAreas_project"
out_coor_syst = arcpy.SpatialReference('WGS 1984')
arcpy.Project_management(input_features, output_feature_class, out_coor_syst)

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb\\communityAreas_project'>

In [34]:
# joining new projected community to data
arcpy.env.qualifiedFieldNames = False
comFc = "comAreasWithAttributes"
joined_table = arcpy.management.AddJoin("communityAreas_project", "areaNUM", "communityDATA", "commNUM")
# Copy the layer to a new permanent feature class
arcpy.CopyFeatures_management(joined_table, comFc)

<Result 'H:/ArcGIS/geogfinalproject/tri.gdb\\comAreasWithAttributes'>

In [45]:
# fieldmapping to spatially join tri points to community areas
targetFeatures = os.path.join(mygdb, comFc)
joinFeatures = os.path.join(mygdb, "TRIpoints")

In [46]:
outfc = os.path.join(mygdb, "communityTRI")

In [47]:
# new fieldmappings
fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(targetFeatures)
fieldmappings.addTable(joinFeatures)

In [48]:
# using the total on site field
totalONSITEFieldIndex = fieldmappings.findFieldMapIndex("totalONSITE")
fieldmap = fieldmappings.getFieldMap(totalONSITEFieldIndex)

In [49]:
# getting the output properties
field = fieldmap.outputField

In [50]:
# naming the sum of points
field.name = "sum_onsite"
field.aliasname = "sum_onsite"
fieldmap.outputField = field

In [51]:
# merging
fieldmap.mergeRule = "sum"
fieldmappings.replaceFieldMap(totalONSITEFieldIndex, fieldmap)

In [96]:
# running the spatial join
arcpy.SpatialJoin_analysis(targetFeatures, joinFeatures, outfc, "#", "#", fieldmappings, "WITHIN_A_DISTANCE", "5000 Meters")

<Result 'H:\\ArcGIS\\geogfinalproject\\tri.gdb\\communityTRI'>